In [ ]:
import cv2
import numpy as np
import torch
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort
import face_recognition
import collections

# Load YOLOv8 model for person detection
model = YOLO("yolov8n.pt")

# Initialize DeepSORT tracker with optimized parameters
tracker = DeepSort(max_age=100)  # Increased max_age to reduce flickering

# Confidence threshold for filtering detections
CONFIDENCE_THRESHOLD = 0.4  

# Dictionaries for face encoding storage and global tracking
face_db = {}
global_person_db = {}

# Track history for bounding box smoothing
track_history = collections.defaultdict(list)

# Load video sources (change as per requirement)
video_sources = [1,2]  # Replace with RTSP URLs for live feeds
caps = [cv2.VideoCapture(src) for src in video_sources]

frame_count = 0  # Counter to optimize face recognition

# Function to extract face encoding from a bounding box
def get_face_encoding(frame, bbox):
    x1, y1, x2, y2 = bbox
    face = frame[y1:y2, x1:x2]
    if face.size == 0:  # Ensure face is valid
        return None
    rgb_face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
    encodings = face_recognition.face_encodings(rgb_face)
    return encodings[0] if encodings else None

# Function to smooth bounding box movements
def smooth_position(track_id, new_bbox):
    track_history[track_id].append(new_bbox)
    if len(track_history[track_id]) > 5:  # Keep last 5 positions
        track_history[track_id].pop(0)
    
    avg_x1 = sum(b[0] for b in track_history[track_id]) // len(track_history[track_id])
    avg_y1 = sum(b[1] for b in track_history[track_id]) // len(track_history[track_id])
    avg_x2 = sum(b[2] for b in track_history[track_id]) // len(track_history[track_id])
    avg_y2 = sum(b[3] for b in track_history[track_id]) // len(track_history[track_id])
    
    return avg_x1, avg_y1, avg_x2, avg_y2

# Function to process a single frame
def process_frame(frame, cam_id):
    global frame_count
    frame_count += 1  # Increment frame counter

    detections = []
    results = model(frame)

    # Extract bounding boxes from YOLO results
    for box, conf, cls in zip(results[0].boxes.xyxy, results[0].boxes.conf, results[0].boxes.cls):
        x1, y1, x2, y2 = map(int, box.tolist())
        conf = float(conf)
        cls = int(cls)

        if cls == 0 and conf > CONFIDENCE_THRESHOLD:  # Detect only people
            detections.append([(x1, y1, x2, y2), conf])

    # Update tracker
    tracks = tracker.update_tracks(detections, frame=frame)

    tracked_info = []
    for track in tracks:
        if not track.is_confirmed():
            continue

        track_id = track.track_id
        x, y, w, h = map(int, track.to_tlwh())  # Convert to bounding box
        x2, y2 = x + w, y + h  # Convert to (x1, y1, x2, y2)

        # Optimize face recognition every 10 frames
        if frame_count % 10 == 0 and track_id not in face_db:
            face_encoding = get_face_encoding(frame, (x, y, x2, y2))
            if face_encoding is not None:
                face_db[track_id] = face_encoding

        # Register person in global tracking database
        if track_id not in global_person_db:
            global_person_db[track_id] = cam_id

        tracked_info.append((x, y, x2, y2, track_id))

    return frame, tracked_info

# Main loop to process video feeds
while True:
    for i, cap in enumerate(caps):
        ret, frame = cap.read()
        if not ret:
            continue

        processed_frame, tracked_objects = process_frame(frame, cam_id=i)

        # Draw bounding boxes with smoothing
        for x1, y1, x2, y2, track_id in tracked_objects:
            x1, y1, x2, y2 = smooth_position(track_id, (x1, y1, x2, y2))
            cv2.rectangle(processed_frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(processed_frame, f"ID {track_id}", (x1, y1 - 5), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        # Display real-time tracking
        cv2.imshow(f"Camera {i+1}", processed_frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):  # Press 'Q' to exit
        break

# Release resources
for cap in caps:
    cap.release()
cv2.destroyAllWindows()
